## Setup

In [156]:
#!pip install openpyxl

In [157]:
import os
import pandas as pd

data_dir = "Data"

### Create ICD Mapping for Later

In [158]:
icd9 = pd.read_excel("Data/ICD-9 Code Descriptions.xlsx")

icd9_dict = icd9.set_index("DIAGNOSIS CODE")["LONG DESCRIPTION"].to_dict()

In [159]:
icd9_dict

{'0010': 'Cholera due to vibrio cholerae',
 '0011': 'Cholera due to vibrio cholerae el tor',
 '0019': 'Cholera, unspecified',
 '0020': 'Typhoid fever',
 '0021': 'Paratyphoid fever A',
 '0022': 'Paratyphoid fever B',
 '0023': 'Paratyphoid fever C',
 '0029': 'Paratyphoid fever, unspecified',
 '0030': 'Salmonella gastroenteritis',
 '0031': 'Salmonella septicemia',
 '00320': 'Localized salmonella infection, unspecified',
 '00321': 'Salmonella meningitis',
 '00322': 'Salmonella pneumonia',
 '00323': 'Salmonella arthritis',
 '00324': 'Salmonella osteomyelitis',
 '00329': 'Other localized salmonella infections',
 '0038': 'Other specified salmonella infections',
 '0039': 'Salmonella infection, unspecified',
 '0040': 'Shigella dysenteriae',
 '0041': 'Shigella flexneri',
 '0042': 'Shigella boydii',
 '0043': 'Shigella sonnei',
 '0048': 'Other specified shigella infections',
 '0049': 'Shigellosis, unspecified',
 '0050': 'Staphylococcal food poisoning',
 '0051': 'Botulism food poisoning',
 '0052': 

#### MS-DRG Codes into Dictionary as well

In [160]:
ms_drgs = pd.read_csv("Data/md-drgs.csv")

md_drgs_dict= ms_drgs.set_index("Code")["Description"].to_dict()

In [161]:
md_drgs_dict

{1: 'Heart Transplant or Implant of Heart Assist System with MCC',
 2: 'Heart Transplant or Implant of Heart Assist System without MCC',
 3: 'ECMO or Tracheostomy with MV >96 Hours or Principal Diagnosis Except Face Mouth and Neck with Major O.R. Procedures',
 4: 'Tracheostomy with MV >96 Hours or Principal Diagnosis Except Face Mouth and Neck without Major O.R. Procedures',
 5: 'Liver Transplant with MCC or Intestinal Transplant',
 6: 'Liver Transplant without MCC',
 7: 'Lung Transplant',
 8: 'Simultaneous Pancreas and Kidney Transplant',
 10: 'Pancreas Transplant',
 11: 'Tracheostomy for Face Mouth and Neck Diagnoses or Laryngectomy with MCC',
 12: 'Tracheostomy for Face Mouth and Neck Diagnoses or Laryngectomy with CC',
 13: 'Tracheostomy for Face Mouth and Neck Diagnoses or Laryngectomy without CC/MCC',
 14: 'Allogeneic Bone Marrow Transplant',
 16: 'Autologous Bone Marrow Transplant with CC/MCC',
 17: 'Autologous Bone Marrow Transplant without CC/MCC',
 18: 'Chimeric Antigen Recep

## Cleaning

### Data Dictionary



| #  | Column Name                         | Description                                      | Notes |
|----|--------------------------------------|--------------------------------------------------|-------|
| 1  | DESYNPUF_ID                         | Beneficiary Code                                |       |
| 2  | CLM_ID                               | Claim ID                                        |       |
| 3  | SEGMENT                              | Claim Line Segment                              |       |
| 4  | CLM_FROM_DT                          | Claims start date                               |       |
| 5  | CLM_THRU_DT                          | Claims end date                                 |       |
| 6  | PRVDR_NUM                            | Provider Institution                            |       |
| 7  | CLM_PMT_AMT                          | Claim Payment Amount                            |       |
| 8  | NCH_PRMRY_PYR_CLM_PD_AMT             | NCH Primary Payer Claim Paid Amount            |       |
| 9  | AT_PHYSN_NPI                         | Attending Physician – National Provider Identifier Number |       |
| 10 | OP_PHYSN_NPI                         | Operating Physician – National Provider Identifier Number |       |
| 11 | OT_PHYSN_NPI                         | Other Physician – National Provider Identifier Number |       |
| 12 | CLM_ADMSN_DT                         | Inpatient admission date                        |       |
| 13 | ADMTNG_ICD9_DGNS_CD                  | Claim Admitting Diagnosis Code                 | Irrelevant for us  |
| 14 | CLM_PASS_THRU_PER_DIEM_AMT           | Claim Pass Thru Per Diem Amount                |       |
| 15 | NCH_BENE_IP_DDCTBL_AMT               | NCH Beneficiary Inpatient Deductible Amount    |       |
| 16 | NCH_BENE_PTA_COINSRNC_LBLTY_AM       | NCH Beneficiary Part A Coinsurance Liability Amount |       |
| 17 | NCH_BENE_BLOOD_DDCTBL_LBLTY_AM       | NCH Beneficiary Blood Deductible Liability Amount |       |
| 18 | CLM_UTLZTN_DAY_CNT                   | Claim Utilization Day Count                    |       |
| 19 | NCH_BENE_DSCHRG_DT                   | Inpatient discharged date                      |       |
| 20 | CLM_DRG_CD                           | Claim Diagnosis Related Group Code             |       |
| 21-30 | ICD9_DGNS_CD_1 – ICD9_DGNS_CD_10  | Claim Diagnosis Code 1 – Claim Diagnosis Code 10 |       |
| 31-36 | ICD9_PRCDR_CD_1 – ICD9_PRCDR_CD_6 | Claim Procedure Code 1 – Claim Procedure Code 6 |       |
| 37-81 | HCPCS_CD_1 – HCPCS_CD_45          | Claim HCPCS Code 1 – Claim HCPCS Code 45       | Removed later as all columns are null       |


In [162]:
df = pd.read_csv(data_dir+"/inpatient.csv")

### Format Dates to be DateTime for later calculations

In [163]:
df['NCH_BENE_DSCHRG_DT'] = pd.to_datetime(df['NCH_BENE_DSCHRG_DT'].astype(str), format='%Y%m%d')
df['CLM_ADMSN_DT'] = pd.to_datetime(df['CLM_ADMSN_DT'].astype(str), format='%Y%m%d')

### Null Checks

In [164]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1332822 entries, 0 to 1332821
Data columns (total 81 columns):
 #   Column                          Non-Null Count    Dtype         
---  ------                          --------------    -----         
 0   DESYNPUF_ID                     1332822 non-null  object        
 1   CLM_ID                          1332822 non-null  int64         
 2   SEGMENT                         1332822 non-null  int64         
 3   CLM_FROM_DT                     1331533 non-null  float64       
 4   CLM_THRU_DT                     1331533 non-null  float64       
 5   PRVDR_NUM                       1332822 non-null  object        
 6   CLM_PMT_AMT                     1332822 non-null  float64       
 7   NCH_PRMRY_PYR_CLM_PD_AMT        1332822 non-null  float64       
 8   AT_PHYSN_NPI                    1319470 non-null  float64       
 9   OP_PHYSN_NPI                    776629 non-null   float64       
 10  OT_PHYSN_NPI                    154635 non

In [165]:
null_counts = df.isna().sum()

print(null_counts[null_counts > 0])

CLM_FROM_DT        1289
CLM_THRU_DT        1289
AT_PHYSN_NPI      13352
OP_PHYSN_NPI     556193
OT_PHYSN_NPI    1178187
                 ...   
HCPCS_CD_41     1332822
HCPCS_CD_42     1332822
HCPCS_CD_43     1332822
HCPCS_CD_44     1332822
HCPCS_CD_45     1332822
Length: 69, dtype: int64


In [166]:
## Drop nulls in ICD9_DGNS_CD_1
df = df.dropna(subset=['ICD9_DGNS_CD_1'])

### Remove Irrelevant Columns

### Notes

* HCPCS_CD_1-HCPCS_CD_45: All of them are null for each value
* ADMTNG_ICD9_DGNS_CD: This is the reason they came in (For ex, shortness of breath). This is not something that matters long term in what we are looking at as we worry more about the condition reason for the stay documented in the ICD9_DGNS_CD_1-10 columns

In [167]:
df = df.drop(columns=['HCPCS_CD_1', 'HCPCS_CD_2','HCPCS_CD_3', 'HCPCS_CD_4', 'HCPCS_CD_5', 'HCPCS_CD_6', 'HCPCS_CD_7',
                      'HCPCS_CD_8', 'HCPCS_CD_9', 'HCPCS_CD_10', 'HCPCS_CD_11', 'HCPCS_CD_12','HCPCS_CD_13', 'HCPCS_CD_14',
                      'HCPCS_CD_15', 'HCPCS_CD_16','HCPCS_CD_17', 'HCPCS_CD_18', 'HCPCS_CD_19', 'HCPCS_CD_20','HCPCS_CD_21', 
                      'HCPCS_CD_22', 'HCPCS_CD_23', 'HCPCS_CD_24','HCPCS_CD_25', 'HCPCS_CD_26', 'HCPCS_CD_27', 'HCPCS_CD_28',
                      'HCPCS_CD_29', 'HCPCS_CD_30', 'HCPCS_CD_31', 'HCPCS_CD_32','HCPCS_CD_33', 'HCPCS_CD_34', 'HCPCS_CD_35', 
                      'HCPCS_CD_36','HCPCS_CD_37', 'HCPCS_CD_38', 'HCPCS_CD_39', 'HCPCS_CD_40','HCPCS_CD_41', 'HCPCS_CD_42', 
                      'HCPCS_CD_43', 'HCPCS_CD_44','HCPCS_CD_45','ADMTNG_ICD9_DGNS_CD'])

## Drop records where there isn't multiple stays for a patient

In [168]:
counts = df['DESYNPUF_ID'].value_counts()
duplicates = counts[counts > 1].index

df_duplicates = df[df['DESYNPUF_ID'].isin(duplicates)]

In [169]:
df_duplicates

,DESYNPUF_ID,CLM_ID,SEGMENT,CLM_FROM_DT,CLM_THRU_DT,PRVDR_NUM,CLM_PMT_AMT,NCH_PRMRY_PYR_CLM_PD_AMT,AT_PHYSN_NPI,OP_PHYSN_NPI,...,ICD9_DGNS_CD_7,ICD9_DGNS_CD_8,ICD9_DGNS_CD_9,ICD9_DGNS_CD_10,ICD9_PRCDR_CD_1,ICD9_PRCDR_CD_2,ICD9_PRCDR_CD_3,ICD9_PRCDR_CD_4,ICD9_PRCDR_CD_5,ICD9_PRCDR_CD_6
1,00016F745862898F,196201177000368,1,20090412.0,20090418.0,3900MB,26000.0,0.0,6.476809e+09,NaN,...,2724,19889,5849,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,00016F745862898F,196661177015632,1,20090831.0,20090902.0,3900HM,5000.0,0.0,6.119985e+08,6.119985e+08,...,NaN,NaN,NaN,NaN,7092.0,6186,V5866,NaN,NaN,NaN
3,00016F745862898F,196091176981058,1,20090917.0,20090920.0,3913XU,5000.0,0.0,4.971603e+09,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,00016F745862898F,196261176983265,1,20100626.0,20100701.0,3900MB,16000.0,0.0,6.408400e+09,1.960860e+09,...,7840,7856,4271,NaN,NaN,E8889,NaN,NaN,NaN,NaN
6,0007F12A492FD25D,196661176963773,1,20080919.0,20080922.0,3400WD,5000.0,0.0,8.956736e+09,6.551008e+09,...,78729,V103,34290,NaN,8659.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1332817,FFFE4E347796686D,992891161660972,1,20080910.0,20080929.0,5000NR,22000.0,0.0,3.058559e+09,4.837333e+09,...,4870,99859,4210,NaN,9904.0,30000,486,NaN,NaN,NaN
1332818,FFFE4E347796686D,992021161651979,1,20081125.0,20081202.0,5000NR,4000.0,0.0,3.058559e+09,NaN,...,34292,3970,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1332819,FFFE4E347796686D,992581161650328,1,20100201.0,20100204.0,5000NR,10000.0,0.0,1.665921e+09,NaN,...,42831,V4588,V4511,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1332820,FFFFEE90E1B4D0DF,992731161641904,1,20080220.0,20080224.0,1100CJ,6000.0,0.0,8.380401e+09,8.380401e+09,...,NaN,NaN,NaN,NaN,5534.0,NaN,NaN,NaN,NaN,NaN


### For now to make this process a little easier, replace the ICD code with descriptions

In [170]:
columns_to_replace = ['ICD9_DGNS_CD_1', 'ICD9_DGNS_CD_2', 'ICD9_DGNS_CD_3', 'ICD9_DGNS_CD_4',
                      'ICD9_DGNS_CD_5', 'ICD9_DGNS_CD_6', 'ICD9_DGNS_CD_7', 'ICD9_DGNS_CD_8',
                      'ICD9_DGNS_CD_9', 'ICD9_DGNS_CD_10']

# Replace ICD9 codes with long descriptions for each column in the list
for column in columns_to_replace:
    df_duplicates[column] = df_duplicates[column].map(icd9_dict)

df_duplicates["ICD9_DGNS_CD_1"]

C:\Users\nicol\AppData\Local\Temp\ipykernel_47584\2880248478.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_duplicates[column] = df_duplicates[column].map(icd9_dict)
C:\Users\nicol\AppData\Local\Temp\ipykernel_47584\2880248478.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_duplicates[column] = df_duplicates[column].map(icd9_dict)
C:\Users\nicol\AppData\Local\Temp\ipykernel_47584\2880248478.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try u

1                       Secondary malignant neoplasm of lung
2                 Vaginal enterocele, congenital or acquired
3          Major depressive affective disorder, single ep...
4          Unspecified hereditary and idiopathic peripher...
6                                   Acute pain due to trauma
                                 ...                        
1332817                            Acute respiratory failure
1332818         Cerebral thrombosis with cerebral infarction
1332819    Subendocardial infarction, initial episode of ...
1332820          Malignant neoplasm of kidney, except pelvis
1332821                 Brachial neuritis or radiculitis NOS
Name: ICD9_DGNS_CD_1, Length: 877883, dtype: object

In [171]:
df_duplicates['CLM_DRG_CD_Description'] = df_duplicates['CLM_DRG_CD'].map(md_drgs_dict)

C:\Users\nicol\AppData\Local\Temp\ipykernel_47584\1823910785.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_duplicates['CLM_DRG_CD_Description'] = df_duplicates['CLM_DRG_CD'].map(md_drgs_dict)


## Subset with just readmissions, any cause

In [185]:
# Sort by Beneficiary ID and Admission Date to make math easier
df_sorted = df_duplicates.sort_values(by=['DESYNPUF_ID', 'CLM_ADMSN_DT'])  

# Calculate the time difference between admissions
df_sorted['ADMISSION_DIFF'] = df_sorted.groupby('DESYNPUF_ID')['CLM_ADMSN_DT'].diff().dt.days

# Calculate the time difference between discharge and the next admission
df_sorted['DISCHARGE_TO_NEXT_ADMISSION'] = (df_sorted.groupby('DESYNPUF_ID')['CLM_ADMSN_DT'].shift(-1) - df_sorted['NCH_BENE_DSCHRG_DT']).dt.days

df_sorted[['DESYNPUF_ID','CLM_ADMSN_DT','ADMISSION_DIFF','ICD9_DGNS_CD_1','ICD9_DGNS_CD_2','CLM_DRG_CD_Description','CLM_DRG_CD']]


,DESYNPUF_ID,CLM_ADMSN_DT,ADMISSION_DIFF,ICD9_DGNS_CD_1,ICD9_DGNS_CD_2,CLM_DRG_CD_Description,CLM_DRG_CD
733454,00000B48BCF4AD29,2008-07-07,NaN,Aortic valve disorders,Hyperosmolality and/or hypernatremia,NaN,281
733455,00000B48BCF4AD29,2008-07-24,17.0,Acute on chronic systolic heart failure,"Anemia, unspecified",NaN,296
733456,00000B48BCF4AD29,2008-08-05,12.0,Diseases of tricuspid valve,Iron deficiency anemia secondary to blood loss...,NaN,263
733457,0000525AB30E4DEF,2008-05-25,NaN,Unspecified intestinal obstruction,Atrial flutter,NaN,349
733458,0000525AB30E4DEF,2008-09-18,116.0,Unspecified vascular insufficiency of intestine,"Dependence on respirator, status",NaN,388
...,...,...,...,...,...,...,...
733451,FFFFBC57C82B76C5,2008-11-06,NaN,"Subendocardial infarction, initial episode of ...","Congestive heart failure, unspecified",NaN,289
733452,FFFFBC57C82B76C5,2008-11-12,6.0,Hemorrhage complicating a procedure,Unspecified essential hypertension,NaN,901
733453,FFFFBC57C82B76C5,2008-11-22,10.0,Coronary atherosclerosis of native coronary ar...,Intermediate coronary syndrome,NaN,238
1332820,FFFFEE90E1B4D0DF,2008-02-20,NaN,"Malignant neoplasm of kidney, except pelvis","Osteoarthrosis, unspecified whether generalize...",NaN,692


In [186]:
df_sorted[['DESYNPUF_ID','CLM_ADMSN_DT','NCH_BENE_DSCHRG_DT','DISCHARGE_TO_NEXT_ADMISSION','ICD9_DGNS_CD_1','ICD9_DGNS_CD_2']]

,DESYNPUF_ID,CLM_ADMSN_DT,NCH_BENE_DSCHRG_DT,DISCHARGE_TO_NEXT_ADMISSION,ICD9_DGNS_CD_1,ICD9_DGNS_CD_2
733454,00000B48BCF4AD29,2008-07-07,2008-07-18,6.0,Aortic valve disorders,Hyperosmolality and/or hypernatremia
733455,00000B48BCF4AD29,2008-07-24,2008-07-26,10.0,Acute on chronic systolic heart failure,"Anemia, unspecified"
733456,00000B48BCF4AD29,2008-08-05,2008-08-20,NaN,Diseases of tricuspid valve,Iron deficiency anemia secondary to blood loss...
733457,0000525AB30E4DEF,2008-05-25,2008-06-05,105.0,Unspecified intestinal obstruction,Atrial flutter
733458,0000525AB30E4DEF,2008-09-18,2008-09-24,NaN,Unspecified vascular insufficiency of intestine,"Dependence on respirator, status"
...,...,...,...,...,...,...
733451,FFFFBC57C82B76C5,2008-11-06,2008-11-11,1.0,"Subendocardial infarction, initial episode of ...","Congestive heart failure, unspecified"
733452,FFFFBC57C82B76C5,2008-11-12,2008-11-14,8.0,Hemorrhage complicating a procedure,Unspecified essential hypertension
733453,FFFFBC57C82B76C5,2008-11-22,2008-11-24,NaN,Coronary atherosclerosis of native coronary ar...,Intermediate coronary syndrome
1332820,FFFFEE90E1B4D0DF,2008-02-20,2008-02-24,75.0,"Malignant neoplasm of kidney, except pelvis","Osteoarthrosis, unspecified whether generalize..."


In [188]:
# For any-cause readmissions, check if the admission is within 30 days of the previous one as we don't realy care about ICD code
df_sorted['ANY_CAUSE_READMISSION'] = df_sorted['DISCHARGE_TO_NEXT_ADMISSION'] <= 30

df_sorted[['DESYNPUF_ID','CLM_ADMSN_DT','NCH_BENE_DSCHRG_DT','ANY_CAUSE_READMISSION','DISCHARGE_TO_NEXT_ADMISSION','ICD9_DGNS_CD_1','ICD9_DGNS_CD_2','CLM_DRG_CD_Description','CLM_DRG_CD']]

,DESYNPUF_ID,CLM_ADMSN_DT,NCH_BENE_DSCHRG_DT,ANY_CAUSE_READMISSION,DISCHARGE_TO_NEXT_ADMISSION,ICD9_DGNS_CD_1,ICD9_DGNS_CD_2,CLM_DRG_CD_Description,CLM_DRG_CD
733454,00000B48BCF4AD29,2008-07-07,2008-07-18,True,6.0,Aortic valve disorders,Hyperosmolality and/or hypernatremia,NaN,281
733455,00000B48BCF4AD29,2008-07-24,2008-07-26,True,10.0,Acute on chronic systolic heart failure,"Anemia, unspecified",NaN,296
733456,00000B48BCF4AD29,2008-08-05,2008-08-20,False,NaN,Diseases of tricuspid valve,Iron deficiency anemia secondary to blood loss...,NaN,263
733457,0000525AB30E4DEF,2008-05-25,2008-06-05,False,105.0,Unspecified intestinal obstruction,Atrial flutter,NaN,349
733458,0000525AB30E4DEF,2008-09-18,2008-09-24,False,NaN,Unspecified vascular insufficiency of intestine,"Dependence on respirator, status",NaN,388
...,...,...,...,...,...,...,...,...,...
733451,FFFFBC57C82B76C5,2008-11-06,2008-11-11,True,1.0,"Subendocardial infarction, initial episode of ...","Congestive heart failure, unspecified",NaN,289
733452,FFFFBC57C82B76C5,2008-11-12,2008-11-14,True,8.0,Hemorrhage complicating a procedure,Unspecified essential hypertension,NaN,901
733453,FFFFBC57C82B76C5,2008-11-22,2008-11-24,False,NaN,Coronary atherosclerosis of native coronary ar...,Intermediate coronary syndrome,NaN,238
1332820,FFFFEE90E1B4D0DF,2008-02-20,2008-02-24,False,75.0,"Malignant neoplasm of kidney, except pelvis","Osteoarthrosis, unspecified whether generalize...",NaN,692


Need to mark the index stays as they are recorded as false so that way we can remove everything else. 10/10 Probably a simpiler way but this works in the end

In [193]:
df_sorted['RELATED'] = df_sorted.groupby('DESYNPUF_ID')['ANY_CAUSE_READMISSION'].shift(1, fill_value=False)

# The last stay in a readmission chain should be flagged as part of the chain if it follows a readmission
df_sorted['RELATED'] = df_sorted['ANY_CAUSE_READMISSION'] | df_sorted['RELATED']

df_cleaned = df_sorted[df_sorted['RELATED']]

In [194]:
df_cleaned[['DESYNPUF_ID','CLM_ADMSN_DT','NCH_BENE_DSCHRG_DT','ANY_CAUSE_READMISSION','DISCHARGE_TO_NEXT_ADMISSION','ICD9_DGNS_CD_1','ICD9_DGNS_CD_2']]

,DESYNPUF_ID,CLM_ADMSN_DT,NCH_BENE_DSCHRG_DT,ANY_CAUSE_READMISSION,DISCHARGE_TO_NEXT_ADMISSION,ICD9_DGNS_CD_1,ICD9_DGNS_CD_2
733454,00000B48BCF4AD29,2008-07-07,2008-07-18,True,6.0,Aortic valve disorders,Hyperosmolality and/or hypernatremia
733455,00000B48BCF4AD29,2008-07-24,2008-07-26,True,10.0,Acute on chronic systolic heart failure,"Anemia, unspecified"
733456,00000B48BCF4AD29,2008-08-05,2008-08-20,False,NaN,Diseases of tricuspid valve,Iron deficiency anemia secondary to blood loss...
999387,0000838E2BBC2ADA,2008-04-17,2008-05-05,True,7.0,Care involving other specified rehabilitation ...,"Anemia, unspecified"
999388,0000838E2BBC2ADA,2008-05-12,2008-05-22,False,NaN,Obstructive chronic bronchitis with (acute) ex...,Unspecified acquired hypothyroidism
...,...,...,...,...,...,...,...
999384,FFFEBB75660834C6,2008-02-10,2008-02-19,True,29.0,Pneumonitis due to inhalation of food or vomitus,"Contusion of face, scalp, and neck except eye(s)"
999385,FFFEBB75660834C6,2008-03-19,2008-03-27,False,NaN,Aortic valve disorders,Pulmonary collapse
733451,FFFFBC57C82B76C5,2008-11-06,2008-11-11,True,1.0,"Subendocardial infarction, initial episode of ...","Congestive heart failure, unspecified"
733452,FFFFBC57C82B76C5,2008-11-12,2008-11-14,True,8.0,Hemorrhage complicating a procedure,Unspecified essential hypertension
